In [125]:
import os
import pyterrier as pt
os.environ["JAVA_HOME"] = "/home/andrew/Java"
os.environ["JVM_PATH"] = '/home/andrew/Java/jre/lib/server/libjvm.so'
if not pt.started():
    pt.init()
import ir_datasets as irds
import pandas as pd
from ir_measures import *
from ir_measures import evaluator

In [126]:
DATASET = "msmarco-passage/trec-dl-2019/judged"
RUN_DIR = '/home/andrew/Documents/Code/Annotation/runs/trec-dl-2019'

In [127]:
dataset = irds.load(DATASET)
original_qrels = pd.DataFrame(dataset.qrels_iter())

In [128]:
all_runs = []
for run in os.listdir(RUN_DIR):
    result = pt.io.read_results(os.path.join(RUN_DIR, run)).rename(columns={'qid': 'query_id', 'docno': 'doc_id'})
    result['runname'] = run
    all_runs.append(result)

all_runs = pd.concat(all_runs)

In [129]:
metrics = [nDCG@10]

In [130]:
evaluate = evaluator(metrics, original_qrels)

In [131]:
all_metrics = []

for run, results in all_runs.groupby('runname'):
    if not run.startswith('dl-19'):
        continue
    for obj in evaluate.iter_calc(results):
        all_metrics.append({
            'runname': run,
            'query_id': obj.query_id,
            'metric': obj.measure,
            'value': obj.value
        })

all_metrics = pd.DataFrame(all_metrics)

# Read Pool

In [132]:
POOL_DIR = '/home/andrew/Documents/Code/Annotation/pooling/pool-pilot-study.jsonl'
pool = pd.read_json(POOL_DIR, lines=True)

In [133]:
pool

,text,query,doc_id,query_id,label
0,Chamber Hypertrophy and Enlargment. In hypertr...,causes of left ventricular hypertrophy,2986227,87181,[]
1,The last common cause of right ventricular hyp...,causes of left ventricular hypertrophy,5197133,87181,[]
2,"Over time, this can cause the heart muscle to ...",causes of left ventricular hypertrophy,5469038,87181,[]
3,"Search for new homes, open houses, recently so...",causes of left ventricular hypertrophy,2396481,87181,[]
4,Ventricular hypertrophy. Ventricular hypertrop...,causes of left ventricular hypertrophy,47212,87181,[]
...,...,...,...,...,...
95,Most providers would have you believe that a l...,what does it mean if your tsh is low,1226400,640502,[]
96,1 A low TSH level with a high FT4 level and a ...,what does it mean if your tsh is low,4359297,640502,[]
97,The four TSH assays showed good agreement at l...,what does it mean if your tsh is low,6728990,640502,[]
98,Low TSH and high FT4 levels are usually seen i...,what does it mean if your tsh is low,6730883,640502,[]


## QREL CHECKS

In [134]:
values = original_qrels.relevance.value_counts()

In [135]:
len(original_qrels) # total number of judgements

9260

In [136]:
values[0] / (values[0] + values[1] + values[2] + values[3]) * 100 # percentage of non-relevant judgements

55.70194384449244

In [137]:
(values[0] + values[1]) / (values[0] + values[1] + values[2] + values[3]) * 100 # percentage of non-relevant and partially relevant judgements

72.9913606911447

# Per QID pseudo-Oracle

In [138]:
# find the best performance by each qid, keep the run name and value

best_performances = all_metrics.groupby(['query_id', 'metric']).apply(lambda x: x.loc[x['value'].idxmax()]).reset_index(drop=True)

In [139]:
# using best performance, find the rankings for each query 
all_rankings = []
for row in best_performances.itertuples():
    ranking = all_runs.loc[all_runs['query_id'] == row.query_id]
    ranking = ranking.loc[ranking.runname==row.runname].copy()
    all_rankings.append(ranking)

all_rankings = pd.concat(all_rankings)

In [140]:
best_performances.value.mean()

0.8309784091185828

# Custom nDCG

In [141]:
# from https://github.com/kmbnw/rank_metrics/blob/master/python/ndcg.py

# Copyright 2016 Krysta M Bouzek
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#    http://www.apache.org/licenses/LICENSE-2.0

#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS,
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#    See the License for the specific language governing permissions and
#    limitations under the License.

import numpy as np

"""
Implementation of normalized discounted cumulative gain.

Handy for testing ranking algorithms.

https://en.wikipedia.org/wiki/Discounted_cumulative_gain
"""

def cum_gain(relevance):
    """
    Calculate cumulative gain.
    This ignores the position of a result, but may still be generally useful.

    @param relevance: Graded relevances of the results.
    @type relevance: C{seq} or C{numpy.array}
    """

    if relevance is None or len(relevance) < 1:
        return 0.0

    return np.asarray(relevance).sum()


def dcg(relevance, alternate=True):
    """
    Calculate discounted cumulative gain.

    @param relevance: Graded and ordered relevances of the results.
    @type relevance: C{seq} or C{numpy.array}
    @param alternate: True to use the alternate scoring (intended to
    place more emphasis on relevant results).
    @type alternate: C{bool}
    """

    if relevance is None or len(relevance) < 1:
        return 0.0

    rel = np.asarray(relevance)
    p = len(rel)

    if alternate:
        # from wikipedia: "An alternative formulation of
        # DCG[5] places stronger emphasis on retrieving relevant documents"

        log2i = np.log2(np.asarray(range(1, p + 1)) + 1)
        return ((np.power(2, rel) - 1) / log2i).sum()
    else:
        log2i = np.log2(range(2, p + 1))
        return rel[0] + (rel[1:] / log2i).sum()


def idcg(relevance, alternate=True):
    """
    Calculate pseudo-ideal discounted cumulative gain (maximum possible DCG).

    @param relevance: Graded and ordered relevances of the results.
    @type relevance: C{seq} or C{numpy.array}
    @param alternate: True to use the alternate scoring (intended to
    place more emphasis on relevant results).
    @type alternate: C{bool}
    """

    if relevance is None or len(relevance) < 1:
        return 0.0

    # guard copy before sort
    rel = np.asarray(relevance).copy()
    rel.sort()
    return dcg(rel[::-1], alternate)

def pidcg(upper_bound, alternate=True):
    if upper_bound is None or len(upper_bound) < 1:
        return 0.0

    # guard copy before sort
    rel = np.asarray(upper_bound).copy()
    return dcg(rel, alternate)


def pndcg(relevance, upper_bound, nranks, alternate=True):
    """
    Calculate pseudo normalized discounted cumulative gain.

    @param relevance: Graded and ordered relevances of the results.
    @type relevance: C{seq} or C{numpy.array}
    @param nranks: Number of ranks to use when calculating NDCG.
    Will be used to rightpad with zeros if len(relevance) is less
    than nranks
    @type nranks: C{int}
    @param alternate: True to use the alternate scoring (intended to
    place more emphasis on relevant results).
    @type alternate: C{bool}
    """
    if relevance is None or len(relevance) < 1:
        return 0.0

    if (nranks < 1):
        raise Exception('nranks < 1')

    rel = np.asarray(relevance)
    pad = max(0, nranks - len(rel))

    # pad could be zero in which case this will no-op
    rel = np.pad(rel, (0, pad), 'constant')

    # now slice downto nranks
    rel = rel[0:min(nranks, len(rel))]

    ideal_dcg = idcg(upper_bound, alternate)
    if ideal_dcg == 0:
        return 0.0

    return dcg(rel, alternate) / ideal_dcg

In [142]:
class ndcg_process():
    def __init__(self, qrels, upper_bounds, cutoff=10, merge = False) -> None:
        self.qrels = qrels
        self.upper_bounds = upper_bounds
        self.cutoff = cutoff
        self.calc = self.calc_merge if merge else self.calc_raw

    def calc_merge(self, qrels, results, upper_bound):
        # get top 10 of results
        qrels = qrels.set_index('doc_id').relevance.to_dict()
        results = results.sort_values('score', ascending=False).head(self.cutoff)
        #filter_upper_bound to contain only the documents in results
        upper_bound = upper_bound.loc[upper_bound['doc_id'].isin(results['doc_id'])].sort_values('score', ascending=False)
        # if any docs are missing from upper bound add them to the end with a score of 0
        missing_docs = results.loc[~results['doc_id'].isin(upper_bound['doc_id'])]
        missing_docs['score'] = 0
        upper_bound = pd.concat([upper_bound, missing_docs])

        result_relevance = np.array([qrels.get(doc_id, 0) for doc_id in results['doc_id']])
        upper_bound_relevance = np.array([qrels.get(doc_id, 0) for doc_id in upper_bound['doc_id']])

        return pndcg(result_relevance, upper_bound_relevance, self.cutoff, False)
    
    def calc_raw(self, qrels, results, upper_bound):
        # get top 10 of results
        qrels = qrels.set_index('doc_id').relevance.to_dict()
        results = results.sort_values('score', ascending=False).head(self.cutoff)
        #filter_upper_bound to contain only the documents in results
        upper_bound = upper_bound.sort_values('score', ascending=False).head(self.cutoff)
        result_relevance = np.array([qrels.get(doc_id, 0) for doc_id in results['doc_id']])
        upper_bound_relevance = np.array([qrels.get(doc_id, 0) for doc_id in upper_bound['doc_id']])

        return pndcg(result_relevance, upper_bound_relevance, self.cutoff)
    
    def iter_calc(self, run):
        for qid, results in run.groupby('query_id'):
            qrels = self.qrels.loc[self.qrels['query_id'] == qid]
            upper_bound = self.upper_bounds.loc[self.upper_bounds['query_id'] == qid]
            yield {'query_id' : qid, 'value' : self.calc(qrels, results, upper_bound)}
    
    def calc_aggregate(self, run):
        all_results = []
        for qid, results in run.groupby('query_id'):
            qrels = self.qrels.loc[self.qrels['query_id'] == qid]
            upper_bound = self.upper_bounds.loc[self.upper_bounds['query_id'] == qid]
            all_results.append(self.calc(qrels, results, upper_bound))
        return  {'value' : sum(all_results) / len(all_results)}

In [143]:
ndcg_compute = ndcg_process(original_qrels, all_rankings, cutoff=10, merge=False)

In [144]:
all_metrics = []

for run, results in all_runs.groupby('runname'):
    for obj in ndcg_compute.iter_calc(results):
        all_metrics.append({
            'runname': run,
            'query_id': obj['query_id'],
            'metric': 'pndcg@10',
            'value': obj['value']
        })

all_metrics = pd.DataFrame(all_metrics)

In [153]:
all_metrics.to_csv('pndcg.csv')

In [145]:
all_rankings.name.unique()

array(['ICT-CKNRM_B50', 'TUA1-1', 'p_exp_rm3_bert', 'TUW19-p1-f',
       'ICT-CKNRM_B', 'idst_bert_p2', 'idst_bert_pr2', 'TUW19-p2-f',
       'TUW19-p3-f', 'srchvrs_ps_run2', 'ICT-BERT2', 'idst_bert_pr1',
       'bm25base_prf_p', 'bm25base_rm3_p', 'ms_duet_passage',
       'idst_bert_p1', 'bm25base_ax_p', 'p_bert', 'bm25tuned_prf_p',
       'srchvrs_ps_run3', 'idst_bert_p3'], dtype=object)

In [146]:
ndcg_compute_merge = ndcg_process(original_qrels, all_rankings, cutoff=10, merge=True)
all_metrics_merge = []

for run, results in all_runs.groupby('runname'):
    for obj in ndcg_compute_merge.iter_calc(results):
        all_metrics_merge.append({
            'runname': run,
            'query_id': obj['query_id'],
            'metric': 'pndcg@10',
            'value': obj['value']
        })

all_metrics_merge = pd.DataFrame(all_metrics_merge)

/tmp/ipykernel_2725046/2813294370.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_docs['score'] = 0
/tmp/ipykernel_2725046/2813294370.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_docs['score'] = 0
/tmp/ipykernel_2725046/2813294370.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

In [152]:
all_metrics_merge.to_csv('pndcg_merge.csv')

In [151]:
all_metrics_merge

,runname,query_id,metric,value
0,colbert_monoelectra-base_msmarco-passage-trec-...,1037798,pndcg@10,0.578338
1,colbert_monoelectra-base_msmarco-passage-trec-...,104861,pndcg@10,1.000000
2,colbert_monoelectra-base_msmarco-passage-trec-...,1063750,pndcg@10,0.853938
3,colbert_monoelectra-base_msmarco-passage-trec-...,1103812,pndcg@10,0.832947
4,colbert_monoelectra-base_msmarco-passage-trec-...,1106007,pndcg@10,0.684764
...,...,...,...,...
8427,tirex_set-encoder-large_msmarco-passage-trec-d...,855410,pndcg@10,1.000000
8428,tirex_set-encoder-large_msmarco-passage-trec-d...,87181,pndcg@10,1.000000
8429,tirex_set-encoder-large_msmarco-passage-trec-d...,87452,pndcg@10,0.865445
8430,tirex_set-encoder-large_msmarco-passage-trec-d...,915593,pndcg@10,0.977617


In [147]:
for row in all_metrics.groupby('runname').mean().itertuples():
    print(row)

Pandas(Index='colbert_monoelectra-base_msmarco-passage-trec-dl-2019-judged.run', value=0.8490375001363796)
Pandas(Index='colbert_monoelectra-large_msmarco-passage-trec-dl-2019-judged.run', value=0.84501440543523)
Pandas(Index='colbert_rankgpt4-turbo_msmarco-passage-trec-dl-2019-judged.run', value=0.8690123187944401)
Pandas(Index='colbert_rankgpt4_msmarco-passage-trec-dl-2019-judged.run', value=0.8408271257315655)
Pandas(Index='colbert_rankgpt4o-full_msmarco-passage-trec-dl-2019-judged.run', value=0.8681026171783593)
Pandas(Index='colbert_rankgpt4o_msmarco-passage-trec-dl-2019-judged.run', value=0.8710606033029763)
Pandas(Index='colbert_rankzephyr_msmarco-passage-trec-dl-2019-judged.run', value=0.8099155749791594)
Pandas(Index='colbert_set-encoder-base_msmarco-passage-trec-dl-2019-judged.run', value=0.8753647239218828)
Pandas(Index='colbert_set-encoder-large_msmarco-passage-trec-dl-2019-judged.run', value=0.872315099907381)
Pandas(Index='dl-19-official-input.ICT-BERT2.gz', value=0.15352

/tmp/ipykernel_2725046/3863578299.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  for row in all_metrics.groupby('runname').mean().itertuples():


In [148]:
all_metrics.groupby('runname').mean()

/tmp/ipykernel_2725046/286102387.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  all_metrics.groupby('runname').mean()


,value
runname,
colbert_monoelectra-base_msmarco-passage-trec-dl-2019-judged.run,0.849038
colbert_monoelectra-large_msmarco-passage-trec-dl-2019-judged.run,0.845014
colbert_rankgpt4-turbo_msmarco-passage-trec-dl-2019-judged.run,0.869012
colbert_rankgpt4_msmarco-passage-trec-dl-2019-judged.run,0.840827
colbert_rankgpt4o-full_msmarco-passage-trec-dl-2019-judged.run,0.868103
...,...
tirex_rankgpt4o-full_msmarco-passage-trec-dl-2019-judged.run,0.804637
tirex_rankgpt4o_msmarco-passage-trec-dl-2019-judged.run,0.798026
tirex_rankzephyr_msmarco-passage-trec-dl-2019-judged.run,0.771088


In [149]:
for row in all_metrics_merge.groupby('runname').mean().itertuples():
    print(row)

Pandas(Index='colbert_monoelectra-base_msmarco-passage-trec-dl-2019-judged.run', value=0.9028464591427567)
Pandas(Index='colbert_monoelectra-large_msmarco-passage-trec-dl-2019-judged.run', value=0.8906646584822532)
Pandas(Index='colbert_rankgpt4-turbo_msmarco-passage-trec-dl-2019-judged.run', value=0.9115080662672628)
Pandas(Index='colbert_rankgpt4_msmarco-passage-trec-dl-2019-judged.run', value=0.902405555933431)
Pandas(Index='colbert_rankgpt4o-full_msmarco-passage-trec-dl-2019-judged.run', value=0.901923571004767)
Pandas(Index='colbert_rankgpt4o_msmarco-passage-trec-dl-2019-judged.run', value=0.9070940896622044)
Pandas(Index='colbert_rankzephyr_msmarco-passage-trec-dl-2019-judged.run', value=0.8896692599666143)
Pandas(Index='colbert_set-encoder-base_msmarco-passage-trec-dl-2019-judged.run', value=0.9103790433225438)
Pandas(Index='colbert_set-encoder-large_msmarco-passage-trec-dl-2019-judged.run', value=0.9104004681349117)
Pandas(Index='dl-19-official-input.ICT-BERT2.gz', value=0.1927

/tmp/ipykernel_2725046/1478244625.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  for row in all_metrics_merge.groupby('runname').mean().itertuples():
